In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("14-15.csv", parse_dates=True, dayfirst=True, index_col=[1])

In [17]:
df = data[["HomeTeam", "AwayTeam", "FTR"]].copy()
df.to_csv("14-15_stripped.csv")

In [4]:
teams = df.HomeTeam.unique()
games = [[]*38 for team in teams]
team_game_map = dict(zip(teams, games))
print(team_game_map)

{'West Brom': [], 'Crystal Palace': [], 'Chelsea': [], 'Stoke': [], 'Man City': [], 'QPR': [], 'West Ham': [], 'Sunderland': [], 'Man United': [], 'Hull': [], 'Southampton': [], 'Liverpool': [], 'Arsenal': [], 'Tottenham': [], 'Leicester': [], 'Everton': [], 'Burnley': [], 'Swansea': [], 'Newcastle': [], 'Aston Villa': []}


In [5]:
def log_score(team, pts):
    '''Write down the running points of a team after game i'''
    
    team_scores = team_game_map[team]
    if len(team_scores) > 0:
        team_game_map[team].append(pts + team_scores[-1])
    else:
        team_game_map[team].append(pts)

for idx, row in df.iterrows():
    home = row.HomeTeam
    away = row.AwayTeam
    if row.FTR == "D":
        log_score(home, 1)
        log_score(away, 1)
    elif row.FTR == "H":
        log_score(home, 3)
        log_score(away, 0)
    else:
        log_score(away, 3)
        log_score(home, 0)

In [6]:
df['HomeTeamCR'] = pd.Series([None]*df.shape[0], index=df.index)
df['AwayTeamCR'] = pd.Series([None]*df.shape[0], index=df.index)

In [13]:
def relegation_stat(tgm):
    '''Calculate closeness to relegation for the points tgm from team_game_map[someteam]'''
    
    out = [0]
    max_pts = 3*38
    
    for idx, pts in enumerate(tgm):
        theo_pts = max_pts - (idx+1)*3
        r = 1 - ((pts + theo_pts) / max_pts)
        out.append(r)
        
    return out

reg_stats = {team: relegation_stat(team_game_map[team]) for team in teams}

In [14]:
def add_releg_status(df):
    team_idx = dict(zip(teams, [0]*len(teams)))
    for idx, row in df.iterrows():
        import pdb
        #pdb.set_trace()
        home = row.HomeTeam
        away = row.AwayTeam
        home_reg_stat = reg_stats[home][team_idx[home]]
        away_reg_stat = reg_stats[away][team_idx[away]]
        team_idx[home] = team_idx[home] + 1
        team_idx[away] = team_idx[away] + 1
        df.set_value(idx, 'HomeTeamCR', home_reg_stat)
        df.set_value(idx, 'AwayTeamCR', away_reg_stat)

add_releg_status(df)

In [15]:
df[:]

,HomeTeam,AwayTeam,FTR,HomeTeamCR,AwayTeamCR
Date,,,,,
2014-08-16,Arsenal,Crystal Palace,H,0,0
2014-08-16,Leicester,Everton,D,0,0
2014-08-16,Man United,Swansea,A,0,0
2014-08-16,QPR,Hull,A,0,0
2014-08-16,Stoke,Aston Villa,A,0,0
2014-08-16,West Brom,Sunderland,D,0,0
2014-08-16,West Ham,Tottenham,A,0,0
2014-08-17,Liverpool,Southampton,H,0,0
2014-08-17,Newcastle,Man City,A,0,0


In [16]:
df.to_csv("14-15_stripped_relegation.csv")